In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
from pyzotero.zotero import Zotero
import os
from sqlite3 import connect

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.dispatcher import Dispatcher
from utils.db import ZoteroDatabase
from utils.scihub import SciHub

In [ ]:
zot = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)
zot.item_template('attachment', 'linked_file')

In [ ]:
keys = ['G3GP7TW8','UYJJPTXG','4SDR5E5R','JTWSQKIS','AG9I49JZ', '7ARSI4KK']
item_type = 'journalArticle || attachment'

z = ZoteroDatabase(local=True)
items = z.get_items_data_local(keys=keys)
z.get_items(keys=keys)
# z.get_items_collections_local()

In [ ]:
conn = connect(os.environ['ZOTERO_DB_PATH'])
c = conn.cursor()

# res = get_items_creators(None, c, item_type='')



c.close()

res

# misc

In [ ]:
z = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)
z.item('UYJJPTXG')['data'];

In [ ]:
z.top()[0]

In [ ]:
z.item_attachment_link_modes()


In [ ]:
z.item_types()
z.item_type_fields('note')
z.item_attachment_link_modes()
# z.item_template('journalArticle')
z.item_template('attachment', 'linked_file')

In [ ]:
conn = connect(os.environ['ZOTERO_DB_PATH'])
c = conn.cursor()

data = get_item_data('UYJJPTXG', local_cursor=c)
# res = get_creators('UYJJPTXG', local_cursor=c)
# res = get_tags('UYJJPTXG', local_cursor=c)
# data = get_collections('UYJJPTXG', local_cursor=c)
# data = get_relations('UYJJPTXG', local_cursor=c)
# items = get_autocompletes(c)
items = get_attachments('UYJJPTXG', c)

c.close()


In [ ]:
items

# scihub

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.scihub import SciHub

In [ ]:
sh = SciHub(use_fallback=True)
sh.available_base_url_list
# result = sh.fetch('http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=1648853')
result = sh.download('10.1016/j.ssmph.2019.100387', path='/home/boris/Downloads/paper.pdf')

In [ ]:
result.keys()

In [ ]:
z = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)

In [ ]:
file_name = 'test_paper.pdf'

att = z.item_template('attachment', 'linked_file')
att['title'] = file_name
att['path'] = 'attachments:' + file_name

z.check_items([att])

In [ ]:
response = z.create_items([att], parentid='AG9I49JZ')
response

In [ ]:
response['success']['0']

# lisc - retrieve DOI relations

In [ ]:
z = Zotero(os.environ['ZOTERO_USER_ID'], 'user', os.environ['ZOTERO_API_KEY'], preserve_json_order=True)
item = z.item('AG9I49JZ')
item2 = z.item('7ARSI4KK')
item['data']['relations'] = {'dc:relation': ['http://zotero.org/users/5832834/items/7ARSI4KK']}
z.check_items([item['data']])
z.update_item(item['data'])

In [ ]:
len(z.items(format='keys', limit=-1))

In [ ]:
def get_items(keys=[], item_type='', tags='', limit=-1, local_cursor=None):
    if not isinstance(keys, list):
        keys = [keys]
    keys_str = ','.join(keys)
    
#     cursor = local_cursor or self.cursor

    if cursor is None:
        self.zot.items(itemKey=keys_str, itemType=item_type, tag=tags, limit=limit)
    else:
        sql = """SELECT
            i.key,
            --i.version,
            MAX(it.typeName) AS itemType,
            MAX(CASE WHEN fieldName = 'title' THEN value END) AS title,
            MAX(CASE WHEN fieldName = 'url' THEN value END) AS url,
            MAX(CASE WHEN fieldName = 'volume' THEN value END) AS volume,
            MAX(CASE WHEN fieldName = 'issue' THEN value END) AS issue,
            MAX(CASE WHEN fieldName = 'pages' THEN value END) AS pages,
            MAX(CASE WHEN fieldName = 'publicationTitle' THEN value END) AS publicationTitle,
            MAX(CASE WHEN fieldName = 'ISSN' THEN value END) AS ISSN,
            MAX(CASE WHEN fieldName = 'date' THEN value END) AS date,
            MAX(CASE WHEN fieldName = 'DOI' THEN value END) AS DOI,
            MAX(CASE WHEN fieldName = 'accessDate' THEN value END) AS accessDate,
            MAX(CASE WHEN fieldName = 'libraryCatalog' THEN value END) AS libraryCatalog,
            MAX(CASE WHEN fieldName = 'language' THEN value END) AS language,
            MAX(CASE WHEN fieldName = 'abstractNote' THEN value END) AS abstractNote,
            MAX(CASE WHEN fieldName = 'shortTitle' THEN value END) AS shortTitle,
            MAX(CASE WHEN fieldName = 'place' THEN value END) AS place,
            MAX(CASE WHEN fieldName = 'publisher' THEN value END) AS publisher,
            MAX(CASE WHEN fieldName = 'ISBN' THEN value END) AS ISBN,
            MAX(CASE WHEN fieldName = 'callNumber' THEN value END) AS callNumber,
            MAX(CASE WHEN fieldName = 'numPages' THEN value END) AS numPages,
            MAX(CASE WHEN fieldName = 'extra' THEN value END) AS extra,
            MAX(CASE WHEN fieldName = 'bookTitle' THEN value END) AS bookTitle,
            MAX(CASE WHEN fieldName = 'journalAbbreviation' THEN value END) AS journalAbbreviation,
            MAX(CASE WHEN fieldName = 'edition' THEN value END) AS edition,
            MAX(CASE WHEN fieldName = 'series' THEN value END) AS series,
            MAX(CASE WHEN fieldName = 'seriesNumber' THEN value END) AS seriesNumber,
            MAX(CASE WHEN fieldName = 'institution' THEN value END) AS institution,
            MAX(CASE WHEN fieldName = 'reportNumber' THEN value END) AS reportNumber,
            MAX(CASE WHEN fieldName = 'conferenceName' THEN value END) AS conferenceName,
            MAX(CASE WHEN fieldName = 'proceedingsTitle' THEN value END) AS proceedingsTitle,
            MAX(CASE WHEN fieldName = 'reportType' THEN value END) AS reportType,
            MAX(CASE WHEN fieldName = 'seriesTitle' THEN value END) AS seriesTitle,
            MAX(CASE WHEN fieldName = 'thesisType' THEN value END) AS thesisType,
            MAX(CASE WHEN fieldName = 'university' THEN value END) AS university,
            MAX(CASE WHEN fieldName = 'archive' THEN value END) AS archive,
            MAX(CASE WHEN fieldName = 'rights' THEN value END) AS rights,
            MAX(CASE WHEN fieldName = 'websiteTitle' THEN value END) AS websiteTitle,
            MAX(CASE WHEN fieldName = 'encyclopediaTitle' THEN value END) AS encyclopediaTitle,
            MAX(CASE WHEN fieldName = 'websiteType' THEN value END) AS websiteType,
            MAX(CASE WHEN fieldName = 'archiveLocation' THEN value END) AS archiveLocation,
            MAX(CASE WHEN fieldName = 'blogTitle' THEN value END) AS blogTitle,
            MAX(CASE WHEN fieldName = 'manuscriptType' THEN value END) AS manuscriptType,
            MAX(dateAdded) AS dateAdded,
            MAX(dateModified) AS dateModified
        FROM items i
        LEFT JOIN itemData id ON i.itemID = id.itemID
        LEFT JOIN itemTypes it ON i.itemTypeID = it.itemTypeID

        LEFT JOIN fields f ON id.fieldID = f.fieldID
        LEFT JOIN itemDataValues idv ON id.valueID = idv.valueID
        WHERE TRUE"""
    sql_keys = "\nAND i.key IN (?)"
    if item_type[0] == '-':
        negate = 'NOT '
        item_type = item_type[1:]
    else:
        negate = ''
    types = map(lambda s: '"{}"'.format(s), item_type.split(' || '))
    sql_type = '\nAND ' + negate + 'IN (' + ','.join(types) + ')'
    sql_group = "\nGROUP BY i.key"
    sql_limit = "\nLIMIT " + str(limit)
    
    if tags != '':
        warnings.warn("tag filter not implemented")
        
    sql += sql_keys if len(keys) > 0
    sql += sql_type if len(item_type) > 0
    sql += sql_group
    sql += sql_limit if limit > 0
    sql += ";"
    
    local_cursor.execute(sql, (key,))
    columns = [c[0] for c in local_cursor.description]
    data = local_cursor.fetchall()
    
    full_item_dict = [dict(zip(columns, item)) for item in data]

    items_list = map(lambda: i: {k:v for k, v in i.items() if v is not None}, full_item_dict)
    return items_list


def execute_add_relations():
    pass

def update_relations(keys, local_read_cursor=None):
    if not isinstance(keys, list):
        keys = [keys]
    
    cursor = read_cursor or 
    
    
update_relations()

In [ ]:
from lisc.requester import Requester
from lisc.urls.open_citations import OpenCitations

util = 'citations' # citations, references
settings = {'format': 'json'}
dois = ['10.1016/S0305-750X(01)00109-7']

urls = OpenCitations()
urls.build_url(util=util)

url = urls.get_url(util=util, segments=dois, settings=settings)
# urls.authenticate(url)
# urls.fill_settings(format='json')
# urls.check_url(util)

req = Requester(wait_time=0.1, logging=None)
# req.check()
r = req.request_url(url)
req.close()

r.json()